# Projeto 1 - Ciência dos Dados

Nome: Enzo Junqueira

Nome: Thomas Rudge

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [22]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [23]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Enzo\OneDrive - Insper - Institudo de Ensino e Pesquisa\Documentos\INSPER\2023\2-SEMESTRE\CIENCIA-DOS-DADOS\Projeto 1\Projeto1_CDADOS


Carregando a base de dados com os tweets classificados manualmente:

In [24]:
treino = pd.read_excel('dados_treino.xlsx')
treino.head()

,Mensagem,"Target: Avaliaremos quais reviews estao relacionados à utilização do Kindle, conteúdo do livro ou com erros de edição do livro (1 = Conteúdo, 2 = Amazon , 3 = Edição)",Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Comprei o livro e ele não baixou. Gostaria de ...,2,NaN,Rótulos de Linha,Contagem de Mensagem
1,"Muita enrolação, pouca informação. A autora fi...",1,NaN,1,190
2,"Não gostei, a capa diz mais do que o conteúdo ...",1,NaN,2,72
3,Estava tentando há dias continuar lendo sem qu...,1,NaN,3,38
4,Incapaz de conviver com ideias diferentes na a...,1,NaN,Total Geral,300


In [25]:
teste = pd.read_excel('dados_teste.xlsx')
teste.head()

,Mensagem,"Target: Avaliaremos quais reviews estao relacionados à utilização do Kindle, conteúdo do livro ou com erros de edição do livro (1 = Conteúdo, 2 = Amazon , 3 = Edição)",Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Como o livro se concentra na política american...,1,NaN,Rótulos de Linha,Contagem de Mensagem
1,"Boa estória. Quis ler antes de ver o filme, qu...",3,NaN,1,131
2,Não gostei. O livro o tempo todo traz narrativ...,1,NaN,2,53
3,Muito confuso.,1,NaN,3,16
4,"Comprei o livro logo após comprar ""Garota Exem...",1,NaN,(vazio),NaN


Renomearemos a coluna de rótulos para algo mais simples, como "Target".
Descartaremos as colunas desnecessárias.
Carregaremos a lista de stop words do arquivo fornecido.

In [26]:
treino.columns = ['Mensagem', 'Target', 'Drop1', 'Drop2', 'Drop3']
teste.columns = ['Mensagem', 'Target', 'Drop1', 'Drop2', 'Drop3']

treino = treino[['Mensagem', 'Target']]
teste = teste[['Mensagem', 'Target']]

In [27]:
with open('stopwords.txt', 'r') as file:
    stopwords = file.read().splitlines()
stopwords

['a, agora, ainda, alguém, algum, alguma, algumas, alguns, ampla, amplas, amplo, amplos, ante, antes, ao, aos, após, aquela, aquelas, aquele, aqueles, aquilo, as, até, através, cada, coisa, coisas, com, como, contra, contudo, da, daquele, daqueles, das, de, dela, delas, dele, deles, depois, dessa, dessas, desse, desses, desta, destas, deste, deste, destes, deve, devem, devendo, dever, deverá, deverão, deveria, deveriam, devia, deviam, disse, disso, disto, dito, diz, dizem, do, dos, e, é, ela, elas, ele, eles, em, enquanto, entre, era, essa, essas, esse, esses, esta, está, estamos, estão, estas, estava, estavam, estávamos, este, estes, estou, eu, fazendo, fazer, feita, feitas, feito, feitos, foi, for, foram, fosse, fossem, grande, grandes, há, isso, isto, já, la, lá, lhe, lhes, lo, mas, me, mesma, mesmas, mesmo, mesmos, meu, meus, minha, minhas, muita, muitas, muito, muitos, na, não, nas, nem, nenhum, nessa, nessas, nesta, nestas, ninguém, no, nos, nós, nossa, nossas, nosso, nossos, num

In [38]:

import re
from unidecode import unidecode

def preprocess_text(text, stopwords):
    """
    Function to preprocess the text data:
    - Remove special characters, numbers, and punctuations
    - Convert text to lowercase
    - Remove stop words
    """

    # Replace accented characters with unaccented characters
    text = ' '.join([unidecode(word) for word in text.split()])

    # Remove all special characters, numbers, and punctuations
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    
    # Convert text to lowercase
    text = text.lower()
    
    # Remove stop words
    text = ' '.join([word for word in text.split() if word not in stopwords])
    
    return text


# Apply the preprocessing to each message in the training and test data
treino['Mensagem'] = treino['Mensagem'].apply(lambda x: preprocess_text(x, stopwords))
teste['Mensagem'] = teste['Mensagem'].apply(lambda x: preprocess_text(x, stopwords))

# Display the first few rows of the preprocessed training and test data
treino.head(), teste.head()


(                                            Mensagem  Target
 0  comprei o livro e ele n o baixou gostaria de s...       2
 1  muita enrola o pouca informa o a autora fica c...       1
 2  n o gostei a capa diz mais do que o conte do d...       1
 3  estava tentando h dias continuar lendo sem que...       1
 4  incapaz de conviver com ideias diferentes na a...       1,
                                             Mensagem  Target
 0  como o livro se concentra na pol tica american...       1
 1  boa est ria quis ler antes de ver o filme que ...       3
 2  n o gostei o livro o tempo todo traz narrativa...       1
 3                                      muito confuso       1
 4  comprei o livro logo ap s comprar garota exemp...       1)

Limpar as mensagens, removendo caracteres especiais, números e pontuações.
Converter todas as palavras para minúsculas para manter a uniformidade.
Remover as stop words.

___
## Classificador automático


Target: Avaliaremos quais reviews estão relacionados à utilização do Kindle, conteúdo do livro ou com erros de edição do livro

1. Kindle: Reviews relacionados à experiência de leitura no dispositivo Kindle.
2. Conteúdo: Reviews que discutem o conteúdo do livro, como enredo, personagens, estilo de escrita, etc.
3. Erros de Edição: Reviews que mencionam erros de edição, como problemas de formatação, erros de impressão...


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**